In [ ]:
import bbcpy
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal
if __name__ == "__main__":
    ival = [1000, 4500]
    band = [10.5, 13]
    imagVPaw = bbcpy.load.eeg.data("./local/data/imagVPaw/imagVPaw.npz")

    #examples for new advanced indexing:

    #select all but some (here exclude all frontal and fronto-central) channels:
    print(imagVPaw['~F*'].shape)
    #can also be combined - selecting all frontal channels except for 7 and 8s e.g.:
    print(imagVPaw[['F?', '~*7', '~*8']].shape)
    print(imagVPaw[['F?,~*7,~*8']].chans)
    #adding advanced slicing with units - e.g. 2s to 5s in steps of 100ms:
    print(imagVPaw['~F*', '2s:5s:100ms'].shape)
    #you may also select single time points:
    print(imagVPaw[:, '2s,5s,100ms'].shape)
    #the same can be separated in a list:
    print(imagVPaw[:, ['2s', '5s', '100ms']].shape)

    cnt_bp = imagVPaw.lfilter(band)
    epo_bp = cnt_bp.epochs(ival)

    #also works on epoched data, also with comma separation, each taken individually
    print(epo_bp[:, ['C?,~*3,~*4', 'FC*,~*3,~*4']].chans)
    print(epo_bp[:, ['C?,~*3,~*4', 'FC*,~*3,~*4'], '2s:4s:100ms'].shape)
    #and also selecting classes
    print(epo_bp[['left']].shape)
    print(epo_bp['left'][:, ['C?,~*3,~*4', 'FC*,~*3,~*4']].shape)
    #changing order of classes
    print(epo_bp[['right', 'left']][:, ['C?,~*3,~*4', 'FC*,~*3,~*4'], '2s:4s:100ms'].shape)

    hilbilly = bbcpy.functions.base.ImportFunc(signal.hilbert, outClass='same', axis=-1)
    ERD1 = np.abs(hilbilly(cnt_bp)).epochs(ival)
    plt.figure()
    plt.subplot(131)
    bbcpy.visual.scalp.map(imagVPaw, ERD1.classmean(0)[0, :, '3s'])
    plt.subplot(132)
    bbcpy.visual.scalp.map(imagVPaw, ERD1.classmean(1)[0, :, '3s'])
    plt.subplot(133)
    bbcpy.visual.scalp.map(imagVPaw, ERD1.classmean(1)[0, :, '3s'] - ERD1.classmean(0)[0, :, '3s'])
    plt.figure()
    plt.subplot(211)
    plt.plot(ERD1.t, np.squeeze(ERD1.classmean()[:, 'C3']).T)
    plt.subplot(212)
    plt.plot(ERD1.t, np.squeeze(ERD1.classmean()[:, 'C4']).T)
    plt.show()